In [2]:
import pandas as pd
import geopandas as gpd

In [111]:
excel = pd.read_excel("/home/eden/Documents/data/resultats-par-niveau-burvot-t2-france-entiere.xlsx", dtype = str)

In [112]:
geojson = gpd.read_file("/home/eden/Documents/data/contour_vote_latest.geojson.geojson")

In [113]:
paris =  gpd.read_file('/home/eden/Documents/data/paris.geojson')

In [114]:
excel.at[7984, 'Code du b.vote'] = "40.1" #ajustement caen

In [115]:
excel['Code du b.vote'] = excel['Code du b.vote'].str.lstrip('0') #enlever les 0 pour correspondre au geojson

In [116]:
#tom + bureau étranger
excel = excel[excel['Code du département'] != 'ZZ']
excel = excel[excel['Code du département'] != 'ZX']
excel = excel[excel['Code du département'] != 'ZW']
excel = excel[excel['Code du département'] != 'ZP']
excel = excel[excel['Code du département'] != 'ZN']
excel.loc[~excel['Code du département'].astype(str).str.contains(r'\d', na=False), 'Code du département'] = '97'
#virer paris
excel = excel[excel['Libellé de la commune'] != 'Paris']

In [117]:
#renommer les unamed de excel
def renameShit():
    to_rename = ['N°Panneau', 'Sexe', 'Nom', 'Prénom', 'Voix', '% Voix/Ins','% Voix/Exp']
    for curr in to_rename:
        excel.rename(columns={curr: curr + '.1'}, inplace=True)
    unnamed = ['Unnamed: 28', 'Unnamed: 29','Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32','Unnamed: 33','Unnamed: 34']
    for i in range(len(to_rename)):
        excel.rename(columns={unnamed[i]: to_rename[i] + '.2'}, inplace=True)
renameShit()

In [118]:
#nouveau df formatted avec juste les colonnes importantes de excel
formatted = pd.DataFrame()
formatted['id_bvote'] = excel['Code du département'] + excel['Code de la commune'] + '_' + excel['Code du b.vote']
nom = ['Libellé de la commune', 'Votants', '% Abs/Ins', '% Voix/Exp.1', '% Voix/Exp.2']
for curr in nom:
    formatted[nom] = excel[nom]
formatted.rename(columns={'% Voix/Exp.1': '% macron_emmanuel'}, inplace=True)
formatted.rename(columns={'% Voix/Exp.2': '% le_pen_marine'}, inplace=True)

In [119]:
paris.columns

Index(['id_bvote', 'type_election', 'annee', 'numero_tour', 'date_tour',
       'circ_bv', 'quartier_bv', 'arr_bv', 'sec_bv', 'num_bureau',
       'nb_procuration', 'nb_inscrit', 'nb_emargement', 'nb_votant',
       'nb_vote_blanc', 'nb_vote_nul', 'nb_exprime', 'macron_emmanuel',
       'le_pen_marine', 'geo_point_2d', 'created_user', 'created_date',
       'last_edited_user', 'last_edited_date', 'st_area_shape',
       'st_perimeter_shape', 'geometry'],
      dtype='object')

In [120]:
paris = paris.drop(columns=['type_election', 'annee', 'numero_tour', 'date_tour', 'circ_bv', 'quartier_bv', 'arr_bv', 
                    'sec_bv', 'num_bureau', 'nb_procuration', 'nb_emargement', 'nb_vote_blanc',
                    'nb_vote_nul', 'geo_point_2d', 'created_user', 'created_date', 'last_edited_user',
                   'last_edited_date', 'st_area_shape', 'st_perimeter_shape'], axis=1)

In [121]:
paris['% Abs/Ins'] = (paris['nb_inscrit'] - paris['nb_votant'])/ paris['nb_inscrit'] * 100
paris['% macron_emmanuel'] = paris['macron_emmanuel']/ paris['nb_exprime'] * 100
paris['% le_pen_marine'] = paris['le_pen_marine']/ paris['nb_exprime'] * 100

In [122]:
paris = paris.drop(['nb_inscrit', 'nb_exprime', 'macron_emmanuel', 'le_pen_marine'], axis=1)

In [123]:
paris.rename(columns={'nb_votant':'Votants'}, inplace=True)
paris.rename(columns={'id_bvote':'id_bv'}, inplace=True)
formatted.rename(columns={'id_bvote':'id_bv'}, inplace=True)

In [124]:
print(geojson.columns)
print(paris.columns)
print(formatted.columns)
#print(excel['Voix.1'][0])

Index(['id_bv', 'geometry'], dtype='object')
Index(['id_bv', 'Votants', 'geometry', '% Abs/Ins', '% macron_emmanuel',
       '% le_pen_marine'],
      dtype='object')
Index(['id_bv', 'Libellé de la commune', 'Votants', '% Abs/Ins',
       '% macron_emmanuel', '% le_pen_marine'],
      dtype='object')


In [125]:
paris['Libellé de la commune'] = 'Paris'

In [126]:
paris.head(15)

,id_bv,Votants,geometry,% Abs/Ins,% macron_emmanuel,% le_pen_marine,Libellé de la commune
0,14-28,1222,"POLYGON ((2.32779 48.83619, 2.32781 48.83619, ...",20.390879,89.714780,10.285220,Paris
1,13-7,1159,"POLYGON ((2.35490 48.83591, 2.35490 48.83593, ...",30.390390,83.471837,16.528163,Paris
2,15-28,1176,"POLYGON ((2.30481 48.84479, 2.30481 48.84477, ...",22.376238,83.946188,16.053812,Paris
3,9-14,1305,"POLYGON ((2.33229 48.88286, 2.33232 48.88285, ...",19.094854,90.560000,9.440000,Paris
4,12-57,1175,"POLYGON ((2.38328 48.84486, 2.38326 48.84493, ...",23.898964,89.633028,10.366972,Paris
5,19-53,916,"POLYGON ((2.37112 48.88948, 2.37110 48.88948, ...",39.095745,85.973398,14.026602,Paris
6,3-10,1318,"POLYGON ((2.35813 48.86340, 2.35811 48.86337, ...",23.549884,88.310649,11.689351,Paris
7,17-58,1054,"POLYGON ((2.28647 48.88694, 2.28652 48.88694, ...",27.510316,76.476578,23.523422,Paris
8,1-5,846,"POLYGON ((2.34594 48.86295, 2.34592 48.86294, ...",22.242647,85.997522,14.002478,Paris
9,15-34,1315,"POLYGON ((2.31663 48.84681, 2.31665 48.84679, ...",20.206311,83.896940,16.103060,Paris


In [127]:
#virer paris de geojson
geojson = geojson[~geojson['id_bv'].str.startswith('75')]

In [128]:
#print(len(geojson))
#print(len(formatted))

67909
67998


In [133]:
#diff = geojson[~geojson['id_bv'].isin(formatted['id_bv'])]

In [134]:
#print(diff.head(25))

In [135]:
paris = paris.drop(['geometry'], axis=1)

In [136]:
#paris.columns

Index(['id_bv', 'Votants', '% Abs/Ins', '% macron_emmanuel', '% le_pen_marine',
       'Libellé de la commune'],
      dtype='object')

In [138]:
formatted = pd.concat([formatted, paris], ignore_index=True)
print(formatted.tail(15))

       id_bv Libellé de la commune Votants  % Abs/Ins % macron_emmanuel  \
69781  15-16                 Paris    1394  20.660216         80.363912   
69782  16-17                 Paris    1145  20.153417         88.808007   
69783  15-49                 Paris    1051  36.109422         76.938987   
69784    2-6                 Paris    1145  21.305842           89.4785   
69785   9-13                 Paris    1192   24.31746         89.664311   
69786  16-23                 Paris    1134  21.793103         83.010157   
69787  18-29                 Paris    1296  21.786361         89.508197   
69788   5-19                 Paris    1109  20.558739         84.555985   
69789  15-56                 Paris    1368  23.232323         82.408131   
69790  15-41                 Paris    1241  23.630769         80.709343   
69791    3-7                 Paris    1170  18.862691         89.892665   
69792  17-22                 Paris     861  27.586207         84.692597   
69793  14-48             

In [ ]:
#formatted.to_csv("formatted.csv")